## Importar Liberías

**Librerías básicas**

In [143]:
import pandas as pd

**Módulos locales**

In [144]:
from Src.cleaner import *
from Src.featureSelection import *

**Librerías de apoyo**

In [145]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline

**Librerías de Machine Learning**

In [146]:
import h2o
h2o.init(nthreads = -1, max_mem_size = 8)

from h2o.automl import H2OAutoML

from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,31 mins 34 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_alex_7b22ja
H2O cluster total nodes:,1
H2O cluster free memory:,7.997 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


## Limpieza de datos

**Ver procedimiento de limpieza detallado en documento "cleaner.ipynb" en este mismo repositorio**

In [147]:
df = pd.read_csv("Inputs/training_dataset.csv")

In [184]:
df.head(3)

,id,0,1,2,3,4,5,6,7,8,9,author
0,0,0.142636,0.431070,0.032912,-0.486796,0.261718,-1.013906,-0.081827,0.330352,0.438262,-0.761088,Ubuntius
1,1,0.019197,0.525284,0.068476,0.166658,0.261718,0.231839,0.635431,0.133209,0.313536,-0.002374,Philippus
2,2,0.093260,-0.087108,-2.268081,-0.164963,0.261718,1.477585,0.635431,-0.370641,0.126447,1.078357,Marcus


In [191]:
class cleanBible(TransformerMixin):
    def __init__(self):
        self.features =["id", "0","1","2","3","4","5","6","7","8","9","author"]
    
    def fit(self, df):
        #Iniciamos limpieza
        X = df[self.features]
        self.cols = self.X.columns
        X = df.drop(columns=["id","author"], axis=1)
        # self.X = pd.get_dummies(X)
        self.X = X
        
        return self
    
    def transform(self, X):
        return self.X

In [192]:
cleaner = cleanBible()
pipe = make_pipeline(*[
    cleaner,
    StandardScaler(),
    #Normalizer(),
])

In [193]:
X = pipe.fit_transform(df)
X= pd.DataFrame(X, columns=cleaner.cols)
pd.options.display.max_columns =28
X.head(3)

AttributeError: 'cleanBible' object has no attribute 'X'

In [ ]:
df_export = X.copy()
df_export.to_csv("Outputs/bible_cleaned(V1).csv", index=False)

## Transformación de dataset

In [ ]:
loan_csv = "Outputs/bible_cleaned(V1).csv"

data = h2o.import_file(loan_csv, header=1) 

In [ ]:
data.shape

In [ ]:
data.head(3)

## Partición de los datos

In [ ]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [ ]:
print(train.nrow)
print(valid.nrow)
print(test.nrow)

## Extracción de Features

In [ ]:
y = 'price'
x = list(data.columns)

x.remove(y)

In [ ]:
x

## Machine Learning!!

In [ ]:
# rf_fit1 = H2ODeepLearningEstimator(model_id='rf_fit1', seed=1)
# rf_fit1.train(x=x, y=y, training_frame=train)


In [ ]:
# gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        # ntrees=500, 
                                        # score_tree_interval=5,     #used for early stopping
                                        # stopping_rounds=3,         #used for early stopping
                                        # stopping_metric='AUC',     #used for early stopping
                                        # stopping_tolerance=0.0005, #used for early stopping
                                        # seed=1)
# gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

In [ ]:
# rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', ntrees=500, nfolds=8, seed=1)
# rf_fit3.train(x=x, y=y, training_frame=data)

In [ ]:
# rf_fit3

## Métricas de rendimiento

In [ ]:
# rf_perf1 = rf_fit1.model_performance(test)
# rf_perf2 = rf_fit2.model_performance(test)
# rf_perf3 = rf_fit3.model_performance(test)

In [ ]:
# print(rf_perf1)

In [ ]:
# print(rf_perf2)

In [ ]:
# print(rf_perf3)

## Auto Machine Learning!!!

**Imposible en mi PC, migramos a Google Colab (ver documento "Colab.ipynb" en el respositorio)**

## Generación de resultado final

**Importación de dataset de testeo**


In [ ]:

data_test = pd.read_csv("Inputs/diamonds_test.csv")

data_test.head(2)


**Limpieza de dataset**

In [ ]:
class cleanDiamonds_test(TransformerMixin):
    def __init__(self):
        self.features =["id","carat","table","depth","y","z","cut","color","clarity","x"]
    
    def fit(self, df):
        #Iniciamos limpieza
        X = df[self.features]
        X.drop(columns=["table","depth","x","z"], axis=1, inplace=True)
        X["cut"] = X["cut"].replace("Premium", 1).replace("Ideal", 2).replace("Very Good", 3).replace("Very Good", 4).replace("Fair", 5).replace("Good", 6)
        X["color"] = X["color"].replace("D", 1).replace("E", 2).replace("F", 3).replace("G", 4).replace("H", 5).replace("I", 6).replace("J", 7)
        X["clarity"] = X["clarity"].replace("IF", 1).replace("VVS1", 2).replace("VVS2", 3).replace("VS1", 4).replace("VS2", 5).replace("SI1", 6).replace("SI2", 7).replace("I1", 8)
        self.X = X
        self.cols = self.X.columns
        return self
    
    def transform(self, X):
        return self.X

In [ ]:
cleaner = cleanDiamonds_test()
pipe = make_pipeline(*[
    cleaner,
    #StandardScaler(),
    #Normalizer(),
])

In [ ]:
X = pipe.fit_transform(data_test)
X= pd.DataFrame(X, columns=cleaner.cols)
pd.options.display.max_columns =28
X.head(3)

In [ ]:
df_export = X.copy()
df_export.to_csv("Outputs/diamonds_cleaned_test(V3).csv", index=False)

**Transformación a dataframe de H2O**

In [ ]:
loan_csv = "Outputs/diamonds_cleaned_test(V1).csv"

data_test_h2o = h2o.import_file(loan_csv) 

In [ ]:
data_test_h2o.head(2)

**Extracción de predicciones**

In [ ]:
y_pred = rf_fit3.predict(test_data=data_test_h2o)

price = y_pred.as_data_frame()

price = price.rename(columns={"predict":"price"})
price.head()

In [ ]:
result = data_test[["id"]].join(price)

result.head()

**Generación de reporte final**

In [ ]:
resultDef = result[["id","price"]]

resultDef.head(2)

In [ ]:
resultDef.to_csv ("Outputs/submission.csv", index = False, header=True)

## Última comprobación final

In [ ]:
df_check = pd.read_csv("Outputs/submission.csv")

In [ ]:
df_check.head()

In [ ]:
data_test.head()